In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import mnist
from keras.utils import to_categorical

/home/ogura/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [83]:
from sklearn.model_selection import train_test_split
(x_train_valid, y_train_valid), (x_test, y_test) = mnist.load_data()
x_train_valid=x_train_valid.reshape(-1, 28,28,1)
X_test=x_test.reshape(-1, 28,28,1)
y_train_valid = to_categorical(y_train_valid, 10)
y_test = to_categorical(y_test, 10)
X_train, X_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid,train_size=0.8)

/home/ogura/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [84]:
seed=1
batchsize=100

In [85]:
train_gen=ImageDataGenerator(rescale=1./255)
valid_gen=ImageDataGenerator(rescale=1./255)
test_gen=ImageDataGenerator(rescale=1./255)

In [86]:
g_train=train_gen.flow(X_train,y_train,seed=seed,batch_size=batchsize)
g_valid=valid_gen.flow(X_valid,y_valid,seed=seed)
g_test=test_gen.flow(X_test,y_test,seed=seed)

In [87]:
from keras.layers import Input,Conv2D,Dropout,MaxPooling2D,Flatten,Dense
from keras.models import Model

def get_cnn():
    inputs = Input((28,28,1))
    kernel_size = (3, 3)
    max_pool_size = (2, 2)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(inputs)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)

    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = Dropout(0.1)(cnn)
    cnn = Conv2D(64, kernel_size, padding='same', activation='relu')(cnn)
    cnn = MaxPooling2D(pool_size=max_pool_size, strides=(2, 2))(cnn)

    fc = Flatten()(cnn)
    fc = Dense(1024, activation='relu')(fc)
    predictions = Dense(10, activation='softmax')(fc)
    model = Model(inputs=inputs, outputs=predictions)

    model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
    return model


In [88]:
model=get_cnn()
history=model.fit_generator(g,epochs=3,steps_per_epoch=len(g_train),validation_data=g_valid,validation_steps=len(g_valid))

Epoch 1/3
480/480 [==============================] - 12s 25ms/step - loss: 0.1483 - acc: 0.9525 - val_loss: 0.0423 - val_acc: 0.9862
Epoch 2/3
480/480 [==============================] - 10s 21ms/step - loss: 0.0464 - acc: 0.9859 - val_loss: 0.0420 - val_acc: 0.9868
Epoch 3/3
480/480 [==============================] - 10s 21ms/step - loss: 0.0347 - acc: 0.9895 - val_loss: 0.0209 - val_acc: 0.9929


In [90]:
model=get_cnn()
history=model.fit(X_train,y_train,epochs=3,batch_size=batchsize,validation_data=(X_valid,y_valid))

Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 12s 247us/step - loss: 0.6164 - acc: 0.8174 - val_loss: 0.0937 - val_acc: 0.9708
Epoch 2/3
48000/48000 [==============================] - 10s 198us/step - loss: 0.0628 - acc: 0.9806 - val_loss: 0.0369 - val_acc: 0.9882
Epoch 3/3
48000/48000 [==============================] - 10s 199us/step - loss: 0.0438 - acc: 0.9867 - val_loss: 0.0450 - val_acc: 0.9858
